In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import functools
import glob
import logging
import numpy as np
import os
import torch
import importlib as imp

from tqdm import tqdm
tprint = tqdm.write

from torch_geometric.data import Data
from torch_geometric.nn.pool import radius_graph
from torch_geometric.transforms import FixedPoints
from tqdm import tqdm
from typing import Callable, Dict, List, Optional, Union

from typing import Callable, Optional, Union

import torch
from torch import Tensor
from torch.nn import Linear

# from torch_geometric.nn.conv import PointNetConv
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.inits import reset
from torch_geometric.typing import (
    Adj,
    OptTensor,
    PairOptTensor,
    PairTensor
)
from torch_geometric.utils import add_self_loops, remove_self_loops
from torch_geometric.nn.norm import BatchNorm
import pytorch_lightning as pl
import pandas as pd

import aegnn
# from aegnn.models.networks.my_fuse import MyConvBNReLU


pl.seed_everything(12345)
device = torch.device('cuda')

torch.set_printoptions(precision=6)

# path = "/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230328183028/epoch=0-step=202.pt" #fuse
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230331161858/epoch=18-step=3856.pt' # fuse, quant test
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230331160055/epoch=12-step=2638.pt' # fuse, pos div 32
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230331164223/epoch=11-step=2435.pt' # fuse, pos abs
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230331230610/epoch=99-step=20299.pt' # quant test
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230331220429/epoch=1-step=405.pt' # quant debug: dpos = 0
path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230405180647/epoch=21-step=4465.pt' # quant debug
model = torch.load(path).to(device)
model.eval()
dm = aegnn.datasets.NCars(batch_size=1, shuffle=False)
dm.setup()
# print(model.model.fuse1.local_nn.weight)
# data_loader = dm.val_dataloader(num_workers=1).__iter__()
data_loader = dm.val_dataloader(num_workers=1)

assert model.model.fuse1.local_nn.bias is None

if isinstance(model, pl.LightningModule):
    nn_model = model._modules['model']
    nn_layers = nn_model._modules
elif isinstance(model, torch.nn.Module):
    nn_layers = model._modules
else:
    raise TypeError(f'The type of model is {type(model)}, not a `torch.nn.Module` or a `pl.LightningModule`')

from copy import deepcopy
unfused_model = deepcopy(model.model)
unfused_model = unfused_model.to(model.device)
unfused_model.eval()

# for key, nn in nn_layers.items():
#     if isinstance(nn, aegnn.models.networks.my_fuse.MyConvBNReLU):
#         # nn_layers[key].module.running_mean = torch.zeros_like(nn_layers[key].module.running_mean)
#         # nn_layers[key].module.running_var = torch.ones_like(nn_layers[key].module.running_var)
#         # nn_layers[key].module.bias = torch.nn.Parameter(torch.zeros_like(nn_layers[key].module.bias))
#         # nn_layers[key].module.weight = torch.nn.Parameter(torch.ones_like(nn_layers[key].module.weight))
#         # nn_layers[key].module.eps = 1e-16
#         nn_layers[key].to_fused()
#         pass
# fused_model = model
# fused_model.eval()
assert model.model.fused is False
assert model.model.quantized is False
model.model.to_fused()
assert model.model.fused is True
assert model.model.quantized is False
fused_model = model
fused_model.eval()

# num_test_samples = 4
num_test_samples = 2460
with torch.no_grad():
    for i, sample in enumerate(tqdm(data_loader, position=1, desc='Samples', total=num_test_samples)):
        torch.cuda.empty_cache()
        if i==num_test_samples: break
        # tprint(f"\nSample {i}, file_id {sample.file_id}:")

        sample = sample.to(model.device)
        tot_nodes = sample.num_nodes


        unfused_test_sample = sample.clone().detach()
        output_unfused = unfused_model.forward(unfused_test_sample)
        y_unfused = torch.argmax(output_unfused, dim=-1)
        # tprint(f'unfused output = {output_unfused}')
        # tprint(f'{unfused_model.fuse1.local_nn.weight}')

        fused_test_sample = sample.clone().detach()
        output_fused = fused_model.forward(fused_test_sample)
        y_fused = torch.argmax(output_fused, dim=-1)
        # tprint(f'  fused output = {output_fused}')
        # tprint(f'{fused_model.model.fuse1.local_nn.weight}')
        # tprint(fused_model.model.fuse1.fused)

        diff = torch.allclose(y_unfused, y_fused)
        if diff is not True:
            print(i)
            print(f'unfused output = {output_unfused}')
            print(f'  fused output = {output_fused}')
        # print(diff)

        if i==3 :  #debug
            print(i)
            print(f'unfused output = {output_unfused}')
            print(f'  fused output = {output_fused}')



/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 12345


3
unfused output = tensor([[ 6.617686, -7.086989]], device='cuda:0')
  fused output = tensor([[ 6.617687, -7.086992]], device='cuda:0')


Samples: 100%|██████████| 2460/2460 [00:32<00:00, 75.08it/s]


In [2]:
from aegnn.models.networks.my_fuse import MyConvBNReLU

for block in fused_model.model.children():
    if isinstance(block,MyConvBNReLU):

        # print(f'x max = {block.obs_x.max_val}')
        # print(f'x min = {block.obs_x.min_val}')
        # x_scale, _ = block.obs_x.calculate_qparams()
        # print(f'x scale = \n{x_scale}')
        tot_x_max = torch.max(block.obs_x.max_val)
        print(f'tot x max = {tot_x_max}')

        # print(f'y max = {block.obs_y.max_val}')
        tot_y_max = torch.max(block.obs_y.max_val)
        print(f'tot y max = {tot_y_max}')

        # print(f'w max = {block.obs_w.max_val}')
        # print(f'w min = {block.obs_w.min_val}')
        # # # w_scale, _ = block.obs_w.calculate_qparams()
        # # # print(f'w scale = \n{w_scale}')
        # tot_w_max = torch.max(block.obs_w.max_val)
        # tot_w_min = torch.min(block.obs_w.min_val)
        # print(f'tot w max = {tot_w_max}')
        # print(f'tot w min = {tot_w_min}')
        tot_wx_max = torch.max(block.obs_w.max_val[:-2])
        tot_wpos_max = torch.max(block.obs_w.max_val[-2:])
        tot_wx_min = torch.min(block.obs_w.min_val[:-2])
        tot_wpos_min = torch.min(block.obs_w.min_val[-2:])
        # print(f'tot wx   max = {tot_wx_max}, min = {tot_wx_min}')
        # print(f'tot wpos max = {tot_wpos_max}, min = {tot_wpos_min}')
        tot_wx_abs_max = torch.maximum(torch.abs(tot_wx_max), torch.abs(tot_wx_min))
        tot_wpos_abs_max = torch.maximum(torch.abs(tot_wpos_max), torch.abs(tot_wpos_min))
        print(f'tot wx   abs max = {tot_wx_abs_max}')
        print(f'tot wpos abs max = {tot_wpos_abs_max}')




        # tot_b_max = torch.max(block.obs_b.max_val)
        # tot_b_min = torch.min(block.obs_b.min_val)
        # print(f'tot b max = {tot_b_max}')
        # print(f'tot b min = {tot_b_min}')
        
        print(f'calibre = {block.calibre}')
        # print(f'x_scale = {block.x_scale}')
        # print(f'y_scale = {block.y_scale}')
        # print(f'w_scale = {block.w_scale}')
        # print(f'b_scale = {block.b_scale}')
        # print(f'M = {block.M}')
        
        print('')

        # print(f'w min = {block.local_nn.weight}')



tot x max = 1.0
tot y max = 1.1586191654205322
tot wx   abs max = 2.749478578567505
tot wpos abs max = 2.7084128856658936
calibre = True

tot x max = 1.1586191654205322
tot y max = 3.460150718688965
tot wx   abs max = 2.4012269973754883
tot wpos abs max = 2.0738189220428467
calibre = True

tot x max = 3.460150718688965
tot y max = 5.682982444763184
tot wx   abs max = 1.4464362859725952
tot wpos abs max = 0.8188993334770203
calibre = True

tot x max = 5.682982444763184
tot y max = 7.878267765045166
tot wx   abs max = 1.2958091497421265
tot wpos abs max = 0.8594822287559509
calibre = True



In [3]:
assert fused_model.model.quantized is False
fused_model.model.quant()

GraphRes(
  (fuse1): MyConvBNReLU(1, 16)
  (fuse2): MyConvBNReLU(16, 32)
  (fuse3): MyConvBNReLU(32, 32)
  (fuse4): MyConvBNReLU(32, 32)
  (pool7): MaxPoolingX(voxel_size=tensor([15.000000, 12.500000], device='cuda:0'), size=64)
  (fc): Linear(in_features=2048, out_features=2, bias=False)
)

In [4]:
assert fused_model.model.quantized is True
assert fused_model.model.fuse1.quantized is True

In [5]:
for block in fused_model.model.children():
    if isinstance(block,MyConvBNReLU):
        
        print(f'quantized = {block.quantized}')
        print(f'x_scale = {block.x_scale}')
        print(f'y_scale = {block.y_scale}')
        print(f'w_scale = {block.w_scale}')
        print(f'dpos_scale = {block.dpos_scale}')
        # print(f'1/dpos_scale = {1/block.dpos_scale}')
        # print(f'1/round(1/dpos_scale) = {(1/torch.round(1/block.dpos_scale))}')
        # print(f'wx_scale = {block.wx_scale}')
        # print(f'wpos_scale = {block.wpos_scale}')
        print(f'b_scale = {block.b_scale}')
        print(f'M = {block.M}')
        m = torch.round(block.M * 2**(20))
        print(f'm = {m}')
        print(f'M<<>> = {2**(-20)*m}')

        # print(f'b_new = {block.b_new}')
        # print(f'b_quant = {block.b_quant}')

        # print(f'w_quant = {block.w_quant}')
        
        print('')

        # print(f'w min = {block.local_nn.weight}')

quantized = True
x_scale = 0.011764707043766975
y_scale = 0.011764707043766975
w_scale = 0.0216494370251894
dpos_scale = 0.0117647061124444
b_scale = 0.0002546992909628898
M = 0.0216494370251894
m = 22701.0
M<<>> = 0.02164936065673828

quantized = True
x_scale = 0.011764707043766975
y_scale = 0.013569219037890434
w_scale = 0.018907299265265465
dpos_scale = 0.0117647061124444
b_scale = 0.0002224388299509883
M = 0.016392897814512253
m = 17189.0
M<<>> = 0.01639270782470703

quantized = True
x_scale = 0.013569219037890434
y_scale = 0.022286206483840942
w_scale = 0.011389262042939663
dpos_scale = 0.013513513840734959
b_scale = 0.00015454339154530317
M = 0.0069344863295555115
m = 7271.0
M<<>> = 0.006934165954589844

quantized = True
x_scale = 0.022286206483840942
y_scale = 0.0308951698243618
w_scale = 0.010203221812844276
dpos_scale = 0.02222222276031971
b_scale = 0.00022739110863767564
M = 0.007360085844993591
m = 7718.0
M<<>> = 0.0073604583740234375



In [6]:
quant_model = model
print('start quant test')
# dm = aegnn.datasets.NCars(batch_size=1, shuffle=False)
# dm.setup()
# print(model.model.fuse1.local_nn.weight)
# data_loader = dm.val_dataloader(num_workers=1).__iter__()
num_test_samples = 2460
# num_test_samples = 12

unfused_correct = 0
quant_correct = 0
with torch.no_grad():
    # for i, sample in enumerate(tqdm(data_loader, position=1, desc='Samples', total=num_test_samples)):
    for i, sample in enumerate(data_loader):
        torch.cuda.empty_cache()
        if i==num_test_samples: break
        # tprint(f"\nSample {i}, file_id {sample.file_id}:")

        sample = sample.to(model.device)
        tot_nodes = sample.num_nodes


        unfused_test_sample = sample.clone().detach()
        output_unfused = unfused_model.forward(unfused_test_sample)
        y_unfused = torch.argmax(output_unfused, dim=-1)
        # print(f'unfused output = {output_unfused}')
        # print(f'x = {unfused_test_sample.x}')
        # tprint(f'{unfused_model.fuse1.local_nn.weight}')
        unfused_hit = torch.allclose(y_unfused, unfused_test_sample.y)
        if unfused_hit: unfused_correct += 1

        quant_test_sample = sample.clone().detach()
        output_quant = quant_model.forward(quant_test_sample)
        y_quant = torch.argmax(output_quant, dim=-1)
        # print(f'  quant output = {output_quant}')
        # print(f'x = {quant_test_sample.x}')
        # tprint(f'{quant_model.model.fuse1.local_nn.weight}')
        # tprint(quant_model.model.fuse1.fused)

        quant_hit = torch.allclose(y_quant, quant_test_sample.y)
        if quant_hit: quant_correct += 1

        # diff = torch.allclose(y_unfused, y_quant)
        # if diff is not True:
        #     print(i)
        #     print(f'unfused output = {output_unfused}')
        #     print(f'  quant output = {output_quant}')
        #     # print(unfused_test_sample.x)
        #     # print(quant_test_sample.x)
        # print(diff)

unfused_acc = unfused_correct / num_test_samples
quant_acc = quant_correct / num_test_samples

print(f'unfused_acc = {unfused_acc}')
print(f'quant_acc = {quant_acc}')

start quant test
unfused_acc = 0.958130081300813
quant_acc = 0.9520325203252032


In [ ]:
a = unfused_model.debug_x2
b = quant_model.model.debug_dqx2
print(a)
print(b)
# same = torch.allclose(a,b)
# same

In [ ]:
for nn in quant_model.model.children():
    if isinstance(nn, MyConvBNReLU):
        w_quant = nn.local_nn.weight
        b_quant = nn.b_quant
        print(w_quant)
        print(b_quant)